In [1]:
import pytezos
from pytezos.contract.interface import ContractInterface

In [2]:
import requests
import json
from dotenv import load_dotenv
import os

# Testing Gas station API

In [3]:
gas_station_api = "http://localhost:8000"
contract_address = "KT1HUdxmgZUw21ED9gqELVvCty5d1ff41p7J"  # permit NFT contract on Ghostnet

In [4]:
alice_key = "edskRpm2mUhvoUjHjXgMoDRxMKhtKfww1ixmWiHCWhHuMEEbGzdnz8Ks4vgarKDtxok7HmrEo1JzkXkdkvyw7Rtw6BNtSd7MJ7"
alice = pytezos.Key.from_encoded_key(alice_key)
ptz = pytezos.pytezos.using("https://ghostnet.tezos.marigold.dev", alice)

We assume that alice has not been already registered as a user. If she was, skip this.

In [5]:
r = requests.post(
    f"{gas_station_api}/users",
    json = {
        "address": alice.public_key_hash(),
        "name": "alice"
    }
)

In [6]:
assert r.status_code == 200

In [7]:
r = requests.get(f"{gas_station_api}/users/{alice.public_key_hash()}")

In [8]:
alice_user = r.json()
alice_user

{'address': 'tz1VSUr8wwNhLAzempoch5d6hLRiTh8Cjcjb',
 'id': '06d44229-4b75-4df5-9bac-df3b53285859',
 'name': 'alice',
 'withdraw_counter': 0}

In [9]:
r = requests.get(f"{gas_station_api}/credits/{alice_user['address']}")
credits = r.json()[0]
assert r.status_code == 200
credits

{'id': '958caaa8-ed25-4c26-a062-42bd78182399',
 'amount': 0,
 'owner_id': '06d44229-4b75-4df5-9bac-df3b53285859'}

In [10]:
register_contract = {
  "address": contract_address,
  "owner_id": alice_user["id"],
  "name": "NFT1",
  "entrypoints": [
    {
      "name": "mint_token",
      "is_enabled": True
    },
    {
      "name": "transfer",
      "is_enabled": True
    },
    {
      "name": "permit",
      "is_enabled": True
    }
  ],
  "credit_id": credits["id"]
}

In [11]:
r = requests.post(
    f"{gas_station_api}/contracts",
    json = register_contract
)

In [12]:
assert r.status_code == 200

In [13]:
api_info = requests.get(
    f"{gas_station_api}/"
).json()

In [14]:
tr = ptz.transaction(destination=api_info["tezos_address"], amount=int(1e6)).send()

In [15]:
credits_deposit = {
  "id": credits["id"],
  "amount": int(1e6),
  "operation_hash": tr.hash(),
  "owner_id": alice_user["id"]
}
r = requests.put(
    f"{gas_station_api}/deposit",
    json = credits_deposit
)

In [16]:
r.status_code

200

In [17]:
r = requests.get(
    f"{gas_station_api}/credits/{alice_user['id']}"
)
r.json()

[{'id': '958caaa8-ed25-4c26-a062-42bd78182399',
  'amount': 1000000,
  'owner_id': '06d44229-4b75-4df5-9bac-df3b53285859'}]

In [18]:
ct1 = ptz.contract(contract_address)

In [19]:
try:
    print(ct1.storage["ledger"][("tz1UtRfZGHjuHe5PJ7QpksEnn1K7BMJ4qVok", 2)]())
except:
    print(0)

14


In [20]:
op_content = ct1.mint_token([{
    "owner": "tz1UtRfZGHjuHe5PJ7QpksEnn1K7BMJ4qVok",
    "token_id": 2,
    "amount_": 1
}]).as_transaction().contents

In [21]:
operation = {
  "sender_address": "tz1UtRfZGHjuHe5PJ7QpksEnn1K7BMJ4qVok",
  "operations": op_content
}
r = requests.post(
    f"{gas_station_api}/operation",
    json=operation
)
r.status_code

200

Once the API has received the operation, we check the balance of the user for this token. After a while, it's showing as updated.

In [24]:
try:
    print(ct1.storage["ledger"][("tz1UtRfZGHjuHe5PJ7QpksEnn1K7BMJ4qVok", 2)]())
except:
    print(0)

15


In [25]:
r = requests.get(
    f"{gas_station_api}/contracts/{contract_address}"
)
contract_db = r.json()

In [26]:
r = requests.post(
    f"{gas_station_api}/condition",
   json={
      "type": "MAX_CALLS_PER_SPONSEE",
      "contract_id": contract_db["id"],
      "entrypoint_id": None,
      "vault_id": credits["id"],
      "max": 1
    }
)

In [27]:
r

<Response [200]>

Now that we have a condition, new users can use the service only a limited number of times. The first time we call `mint_token` with the same address, everything should work, as the condition is not retroactive.

In [29]:
operation = {
  "sender_address": "tz1UtRfZGHjuHe5PJ7QpksEnn1K7BMJ4qVok",
  "operations": op_content
}
r = requests.post(
    f"{gas_station_api}/operation",
    json=operation
)
r.status_code

200

But the second time it will fail:

In [30]:
operation = {
  "sender_address": "tz1UtRfZGHjuHe5PJ7QpksEnn1K7BMJ4qVok",
  "operations": op_content
}
r = requests.post(
    f"{gas_station_api}/operation",
    json=operation
)
r.status_code

400

However, for another user it still works. Note that we only changed sender_address, and kept op_content the same, so the user receiving the new NFT is still the same.

In [31]:
operation = {
  "sender_address": "tz1aH7Hj1s95wkPjCbMr2RMgTPq4RPHE1LLU",
  "operations": op_content
}
r = requests.post(
    f"{gas_station_api}/operation",
    json=operation
)
r.status_code

200

In [32]:
requests.get(
    f"{gas_station_api}/condition/{credits['id']}"
).json()

[{'type': 'MAX_CALLS_PER_SPONSEE',
  'vault_id': '958caaa8-ed25-4c26-a062-42bd78182399',
  'created_at': '2024-03-12T18:00:11.362107+00:00',
  'id': '928a842a-68da-48c6-b9fa-cccd71ccacb1',
  'contract_id': '4b0a9fdf-d36a-4ce8-af4c-1facc8ae1371',
  'entrypoint_id': None,
  'max': 1,
  'current': 2,
  'is_active': True}]